In [1]:
import sys
sys.path.insert(0, "/Users/namirjawdat/Projects/manipulation")
import manipulation

In [2]:
# from pydrake.geometry.render import RenderLabel
import copy
from functools import partial
import matplotlib.pyplot as plt
import mpld3
import numpy as np
import os
import pydot
import time
from IPython.display import SVG, HTML, display, Markdown

from pydrake.all import (
	AddMultibodyPlantSceneGraph,
	AddUnitQuaternionConstraintOnPlant,
  AddMultibodyPlant,
  AddWeld,
	AutoDiffXd,
  ComInPolyhedronConstraint,
	DiagramBuilder,
	ExtractGradient,
	ExtractValue,
	InitializeAutoDiff,
	JacobianWrtVariable,
	JointIndex,
	MathematicalProgram,
	MeshcatVisualizer,
	MultibodyPlant,
	OrientationConstraint,
	Parser,
	PidController,
	PiecewisePolynomial,
	PositionConstraint,
  Rgba,
	RigidTransform,
  RollPitchYaw,
	RotationMatrix,
	Simulator,
	SnoptSolver,
	Solve,
  Sphere,
	StartMeshcat,
  ToLatex,
  LogVectorOutput,
	eq,
	namedview,
	ModelVisualizer,
	GraphOfConvexSets,
	GraphOfConvexSetsOptions,
	HPolyhedron,
	Point,
  VPolytope,
)

# from manipulation.meshcat_utils import MeshcatPoseSliders, WsgButton

from underactuated import ConfigureParser, multibody, running_as_notebook

In [3]:
# Start the visualizer (run this cell only once, each instance consumes a port)
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7001


The following section contains few supporting functions

In [4]:
def set_home(plant, context):
  PositionView = namedview(
      "Positions",
      plant.GetPositionNames(always_add_suffix=False, add_model_instance_prefix=True),
  )
  positionFields = PositionView.get_fields()
  hip_roll = -0.0
  front_hip_pitch = -0.6
  rear_hip_pitch = -0.9
  knee = 1.55
  q0 = PositionView(np.zeros(len(positionFields)))
  q0.spot_front_right_hip_roll = -hip_roll
  q0.spot_front_right_hip_pitch = front_hip_pitch
  q0.spot_front_right_knee = knee
  q0.spot_front_left_hip_roll = hip_roll
  q0.spot_front_left_hip_pitch = front_hip_pitch
  q0.spot_front_left_knee = knee
  q0.spot_rear_right_hip_roll = -hip_roll
  q0.spot_rear_right_hip_pitch = rear_hip_pitch
  q0.spot_rear_right_knee = knee
  q0.spot_rear_left_hip_roll = hip_roll
  q0.spot_rear_left_hip_pitch = rear_hip_pitch
  q0.spot_rear_left_knee = knee
  q0.spot_arm_shoulder_roll = 0*np.pi/180
  q0.spot_arm_shoulder_pitch = 0*np.pi/180
  q0.spot_arm_elbow_pitch = 0*np.pi/180
  q0.spot_arm_lower_arm_roll = 0*np.pi/180
  q0.spot_arm_wrist_pitch = 0*np.pi/180
  q0.spot_arm_wrist_roll = 160*np.pi/180
  q0.spot_arm_finger_pitch = 0*np.pi/180
  q0.spot_world_body_qw = 1.0  # Unit quaternion
  q0.spot_world_body_z =0.528
  plant.SetPositions(context, q0[:])

def add_spot_arm_sliders(x0):
  meshcat.AddButton("Stop")
  meshcat.AddSlider(
    "shoulder_roll",
    0,
    5.759586531581288,
    1e-6,
    x0[12]
  )

  meshcat.AddSlider(
    "shoulder_pitch",
    0,
    3.665191429188092,
    1e-6,
    x0[13]
  )

  meshcat.AddSlider(
    "elbow_pitch",
    0,
    3.141592653589793,
    1e-6,
    x0[14]
  )

  meshcat.AddSlider(
    "lower_arm_roll",
    -2.792526803190927,
    2.792526803190927,
    1e-6,
    x0[15]
  )

  meshcat.AddSlider(
    "wrist_pitch",
    -1.570796,
    1.570796,
    1e-6,
    x0[16]
  )

  meshcat.AddSlider(
    "wrist_roll",
    -2.879793265790644,
    2.879793265790644,
    1e-6,
    x0[17]
  )

  meshcat.AddSlider(
    "finger_pitch",
    0,
    1.570796326794897,
    1e-6,
    x0[18]
  )

def set_qf(qf_view, command_increment, shoulder_roll_cmd, shoulder_pitch_cmd, elbow_pitch_cmd, lower_arm_roll_cmd, wrist_pitch_cmd, wrist_roll_cmd, finger_pitch_cmd):
  if qf_view.spot_arm_shoulder_roll > shoulder_roll_cmd:
    qf_view.spot_arm_shoulder_roll -= command_increment
    if qf_view.spot_arm_shoulder_roll < shoulder_roll_cmd:
      qf_view.spot_arm_shoulder_roll = shoulder_roll_cmd
  
  if qf_view.spot_arm_shoulder_roll < shoulder_roll_cmd:
    qf_view.spot_arm_shoulder_roll += command_increment
    if qf_view.spot_arm_shoulder_roll > shoulder_roll_cmd:
      qf_view.spot_arm_shoulder_roll = shoulder_roll_cmd


  if qf_view.spot_arm_shoulder_pitch > shoulder_pitch_cmd:
    qf_view.spot_arm_shoulder_pitch -= command_increment
    if qf_view.spot_arm_shoulder_pitch < shoulder_pitch_cmd:
      qf_view.spot_arm_shoulder_pitch = shoulder_pitch_cmd
  
  if qf_view.spot_arm_shoulder_pitch < shoulder_pitch_cmd:
    qf_view.spot_arm_shoulder_pitch += command_increment
    if qf_view.spot_arm_shoulder_pitch > shoulder_pitch_cmd:
      qf_view.spot_arm_shoulder_pitch = shoulder_pitch_cmd

  if qf_view.spot_arm_elbow_pitch > elbow_pitch_cmd:
    qf_view.spot_arm_elbow_pitch -= command_increment
    if qf_view.spot_arm_elbow_pitch < elbow_pitch_cmd:
      qf_view.spot_arm_elbow_pitch = elbow_pitch_cmd

  if qf_view.spot_arm_elbow_pitch < elbow_pitch_cmd:
    qf_view.spot_arm_elbow_pitch += command_increment
    if qf_view.spot_arm_elbow_pitch > elbow_pitch_cmd:
      qf_view.spot_arm_elbow_pitch = elbow_pitch_cmd

  if qf_view.spot_arm_lower_arm_roll > lower_arm_roll_cmd:
    qf_view.spot_arm_lower_arm_roll -= command_increment
    if qf_view.spot_arm_lower_arm_roll < lower_arm_roll_cmd:
      qf_view.spot_arm_lower_arm_roll = lower_arm_roll_cmd

  if qf_view.spot_arm_lower_arm_roll < lower_arm_roll_cmd:
    qf_view.spot_arm_lower_arm_roll += command_increment
    if qf_view.spot_arm_lower_arm_roll > lower_arm_roll_cmd:
      qf_view.spot_arm_lower_arm_roll = lower_arm_roll_cmd

  if qf_view.spot_arm_wrist_pitch > wrist_pitch_cmd:
    qf_view.spot_arm_wrist_pitch -= command_increment
    if qf_view.spot_arm_wrist_pitch < wrist_pitch_cmd:
      qf_view.spot_arm_wrist_pitch = wrist_pitch_cmd

  if qf_view.spot_arm_wrist_pitch < wrist_pitch_cmd:
    qf_view.spot_arm_wrist_pitch += command_increment
    if qf_view.spot_arm_wrist_pitch > wrist_pitch_cmd:
      qf_view.spot_arm_wrist_pitch = wrist_pitch_cmd

  if qf_view.spot_arm_wrist_roll > wrist_roll_cmd:
    qf_view.spot_arm_wrist_roll -= command_increment
    if qf_view.spot_arm_wrist_roll < wrist_roll_cmd:
      qf_view.spot_arm_wrist_roll = wrist_roll_cmd

  if qf_view.spot_arm_wrist_roll < wrist_roll_cmd:
    qf_view.spot_arm_wrist_roll += command_increment
    if qf_view.spot_arm_wrist_roll > wrist_roll_cmd:
      qf_view.spot_arm_wrist_roll = wrist_roll_cmd

  if qf_view.spot_arm_finger_pitch > finger_pitch_cmd:
    qf_view.spot_arm_finger_pitch -= command_increment
    if qf_view.spot_arm_finger_pitch < finger_pitch_cmd:
      qf_view.spot_arm_finger_pitch = finger_pitch_cmd

  if qf_view.spot_arm_finger_pitch < finger_pitch_cmd:
    qf_view.spot_arm_finger_pitch += command_increment
    if qf_view.spot_arm_finger_pitch > finger_pitch_cmd:
      qf_view.spot_arm_finger_pitch = finger_pitch_cmd
  return qf_view

def create_pid_controller(builder, plant):
  num_actuators = plant.num_actuators()
  kp = 300.0 * np.ones(num_actuators)
  ki = 3.0 * np.ones(num_actuators)
  kd = 30.0 * np.ones(num_actuators)
  # Select the joint states (and ignore the floating-base states)
  num_q = plant.num_positions()
  num_q_dot = plant.num_velocities()
  S = np.zeros((2*num_actuators, (num_q + num_q_dot)))

  j = 0
  for jointIdx in range(plant.num_joints()):
    joint = plant.get_joint(JointIndex(jointIdx))
    if joint.num_positions() != 1:
      print(f'Joint[{jointIdx}] {joint.name()} is not considered')
      continue
    # print(f'Joint[{jointIdx}] {joint.name()} is considered')
    # print(f'{joint.name()} position start {joint.position_start()} and velocity start {joint.velocity_start()}')
    S[j, joint.position_start()] = 1
    S[num_actuators + j, num_q + joint.velocity_start()] = 1
    # use lower gain for the knee joints
    if "knee" in joint.name():
      kd[j] = kd[0]
    j = j + 1

  controller = builder.AddSystem(
    PidController(
      kp=kp,
      ki=ki,
      kd=kd,
      state_projection=S,
      output_projection=plant.MakeActuationMatrix()[6:].T,
    )
  )

  return controller, S

# Need this because a==b returns True even if a = AutoDiffXd(1, [1, 2]), b= AutoDiffXd(2, [3, 4])
# That's the behavior of AutoDiffXd in C++, also.
def autoDiffArrayEqual(a, b):
  return np.array_equal(a, b) and np.array_equal(
    ExtractGradient(a), ExtractGradient(b)
  )

Here is a urdf description for a ball to be welded to the finger of the arm to add weight representing the arm carrying an object

In [5]:
ball_mass = 1
ball_radius = 0.075
weight_ball = f"""
  <robot name="simple_ball">
    <link name="ball">
      <inertial>
        <mass value="{ball_mass}" />
        <!-- center of mass (com) is defined w.r.t. link local coordinate system -->
        <!-- give a little y-offset to make the top precess -->
        <origin xyz="0 0 0" /> 
        <inertia  ixx="{2*ball_mass*(ball_radius**2)/5}" ixy="0.0"  ixz="0.0"  iyy="{2*ball_mass*(ball_radius**2)/5}"  iyz="0.0"  izz="{2*ball_mass*(ball_radius**2)/5}" />
      </inertial>
      <visual>
        <!-- visual origin is defined w.r.t. link local coordinate system -->
        <origin xyz="0 0 0" rpy="0 0 0" />
        <geometry>
          <sphere radius="{ball_radius}"/>
        </geometry>
        <material>
          <color rgba="0.0 1.0 0.0 1.0" />
        </material>
      </visual>
      <collision>
        <!-- collision origin is defined w.r.t. link local coordinate system -->
        <origin xyz="0 0 0" rpy="0 0 0" />
        <geometry>
          <sphere radius="{ball_radius}"/>
        </geometry>
      </collision>
    </link>
  </robot>
"""

The following function contains the mathematical program. It will be called periodically. This will cause the mathematical program and the solver to be slow.
my main reason to have it here is to help me isolate the work and concentrate on the formulation of QP rather than speed.

In [6]:
def qp_optimization_continuous(sim_context, timeStep, plant, q0, v0, qf):
  
  prog = MathematicalProgram()
  nq = plant.num_positions()
  nv = plant.num_velocities()
  # Joint positions and velocities
  q = prog.NewContinuousVariables(nq, "q")
  v = prog.NewContinuousVariables(nv, "v")
  vdot = prog.NewContinuousVariables(nv, "vdot")

  PositionView = namedview(
      "Positions", plant.GetPositionNames(always_add_suffix=False, add_model_instance_prefix=True)
  )
  
  VelocityView = namedview(
      "Velocities",
      plant.GetVelocityNames(always_add_suffix=False, add_model_instance_prefix=True),
  )
  q_hip_roll = 9e-4
  q_hip_pitch = 5e-3
  q_knee = 5e-2
  q_arm = 30
  q_world_body = 0
  q_view = PositionView(q)
  v_view = VelocityView(v)
  q_cost = PositionView([1] * nq)
  v_cost = VelocityView([1] * nv)
  q_cost.spot_world_body_x = q_world_body
  q_cost.spot_world_body_y = q_world_body
  q_cost.spot_world_body_z = q_world_body
  q_cost.spot_world_body_qx = q_world_body
  q_cost.spot_world_body_qy = q_world_body
  q_cost.spot_world_body_qz = q_world_body
  q_cost.spot_world_body_qw = q_world_body
  q_cost.spot_front_left_hip_roll = q_hip_roll
  q_cost.spot_front_left_hip_pitch = q_hip_pitch
  q_cost.spot_front_left_knee = q_knee
  q_cost.spot_front_right_hip_roll = q_hip_roll
  q_cost.spot_front_right_hip_pitch = q_hip_pitch
  q_cost.spot_front_right_knee = q_knee
  q_cost.spot_rear_left_hip_roll = q_hip_roll
  q_cost.spot_rear_left_hip_pitch = q_hip_pitch
  q_cost.spot_rear_left_knee = q_knee
  q_cost.spot_rear_right_hip_roll = q_hip_roll
  q_cost.spot_rear_right_hip_pitch = q_hip_pitch
  q_cost.spot_rear_right_knee = q_knee
  q_cost.spot_arm_shoulder_roll = q_arm
  q_cost.spot_arm_shoulder_pitch = q_arm
  q_cost.spot_arm_elbow_pitch = q_arm
  q_cost.spot_arm_lower_arm_roll = q_arm
  q_cost.spot_arm_wrist_pitch = q_arm
  q_cost.spot_arm_wrist_roll = q_arm
  q_cost.spot_arm_finger_pitch = q_arm
  v_cost.spot_world_body_vx = q_world_body/10
  v_cost.spot_world_body_vy = q_world_body/10
  v_cost.spot_world_body_vz = q_world_body/10
  v_cost.spot_world_body_wx = q_world_body/10
  v_cost.spot_world_body_wy = q_world_body/10
  v_cost.spot_world_body_wz = q_world_body/10
  v_cost.spot_front_left_hip_roll = q_hip_roll/10
  v_cost.spot_front_left_hip_pitch = q_hip_pitch/10
  v_cost.spot_front_left_knee = q_knee/10
  v_cost.spot_front_right_hip_roll = q_hip_roll/10
  v_cost.spot_front_right_hip_pitch = q_hip_pitch/10
  v_cost.spot_front_right_knee = q_knee/10
  v_cost.spot_rear_left_hip_roll = q_hip_roll/10
  v_cost.spot_rear_left_hip_pitch = q_hip_pitch/10
  v_cost.spot_rear_left_knee = q_knee/10
  v_cost.spot_rear_right_hip_roll = q_hip_roll/10
  v_cost.spot_rear_right_hip_pitch = q_hip_pitch/10
  v_cost.spot_rear_right_knee = q_knee/10
  v_cost.spot_arm_shoulder_roll = q_arm/10
  v_cost.spot_arm_shoulder_pitch = q_arm/10
  v_cost.spot_arm_elbow_pitch = q_arm/10
  v_cost.spot_arm_lower_arm_roll = q_arm/10
  v_cost.spot_arm_wrist_pitch = q_arm/10
  v_cost.spot_arm_wrist_roll = q_arm/10
  v_cost.spot_arm_finger_pitch = q_arm/10

  plant_context = plant.GetMyContextFromRoot(sim_context)
  q0 = plant.GetPositions(plant_context)
  v0 = plant.GetVelocities(plant_context)
  q0_view = PositionView(q0)
  v0_view = VelocityView(v0)
  # q0_view.spot_front_left_hip_roll = 0
  # q0_view.spot_front_right_hip_roll = 0
  # q0_view.spot_rear_left_hip_roll = 0
  # q0_view.spot_rear_right_hip_roll = 0

  # v0_view.spot_front_left_hip_roll = 0
  # v0_view.spot_front_right_hip_roll = 0
  # v0_view.spot_rear_left_hip_roll = 0
  # v0_view.spot_rear_right_hip_roll = 0

  prog.SetInitialGuess(
    v, v0_view[:]
  ) 
  # Initial guess for all joint angles is the home position
  prog.SetInitialGuess(
    q, q0_view[:]
  )  # Solvers get stuck if the quaternion is initialized with all zeros.

  foot_frame = [
    plant.GetFrameByName("front_left_foot_center"),
    plant.GetFrameByName("front_right_foot_center"),
    plant.GetFrameByName("rear_left_foot_center"),
    plant.GetFrameByName("rear_right_foot_center"),
  ]
   
  # Joint limits
  prog.AddBoundingBoxConstraint(
    plant.GetPositionLowerLimits(),
    plant.GetPositionUpperLimits(),
    q,
  )
  # Joint velocity limits
  prog.AddBoundingBoxConstraint(
    plant.GetVelocityLowerLimits(),
    plant.GetVelocityUpperLimits(),
    v,
  )
  # Unit quaternions
  AddUnitQuaternionConstraintOnPlant(plant, q, prog)

  # # Body orientation
  # constraintOrientation = prog.AddConstraint(
  #   OrientationConstraint(
  #     plant,
  #     body_frame,
  #     RotationMatrix(),
  #     plant.world_frame(),
  #     RotationMatrix(),
  #     0.1,
  #     plant_context,
  #   ),
  #   q,
  # )
  # constraintOrientation.evaluator().set_description("Orientation constraint")

  # Running costs:
  constraint10 = prog.AddQuadraticErrorCost(np.diag(q_cost), qf, q)
  constraint10.evaluator().set_description("position error cost constraint")
  constraint11 = prog.AddQuadraticErrorCost(np.diag(v_cost), [0] * nv, v)
  constraint11.evaluator().set_description("velocity error cost constraint")
  # constraint12 = prog.AddQuadraticErrorCost(np.diag(np.ones(nv)), [0] * nv, vdot)
  # constraint12.evaluator().set_description("velocity dot error cost constraint")
  #############################################################
  #############################################################
  mu = 1  # rubber on rubber
  total_mass = plant.CalcTotalMass(plant.CreateDefaultContext())
  gravity = plant.gravity_field().gravity_vector()

  # standing on all four feet
  in_stance = np.ones(4)
  
  # Contact forces
  contact_force = [
    prog.NewContinuousVariables(3, f"foot{foot}_contact_force")
    for foot in range(4)
  ]

  for foot in range(4):
    # Linear friction cone
    prog.AddLinearConstraint(
      contact_force[foot][0] <= mu * contact_force[foot][2]
    )
    prog.AddLinearConstraint(
      -contact_force[foot][0] <= mu * contact_force[foot][2]
    )
    prog.AddLinearConstraint(
      contact_force[foot][1] <= mu * contact_force[foot][2]
    )
    prog.AddLinearConstraint(
      -contact_force[foot][1] <= mu * contact_force[foot][2]
    )
    # normal force >=0, normal_force == 0 if not in_stance
    prog.AddBoundingBoxConstraint(
      0,
      in_stance[foot] * 4 * 9.81 * total_mass,
      contact_force[foot][2],
    )

  foot_pose_in_world = np.zeros((4,2))
  for idx in range(4):
    foot_pose_in_world[idx, :] = plant.CalcPointsPositions(
      plant_context, foot_frame[idx], [0, 0, 0], plant.world_frame()
    )[:2,0]

  #############################################################
  #############################################################
  for i in range(4):
    if in_stance[i]:
      # foot should be on the ground (world position z=0)
      prog.AddConstraint(
        PositionConstraint(
          plant,
          plant.world_frame(),
          [foot_pose_in_world[i, 0], foot_pose_in_world[i, 1], 0],
          [foot_pose_in_world[i, 0], foot_pose_in_world[i, 1], 0],
          foot_frame[i],
          [0, 0, 0],
          plant_context,
        ),
        q,
      )
  #############################################################
  #############################################################

  V = VPolytope(foot_pose_in_world.T)
  halfspace_polyhedral = HPolyhedron(V)

  # Center of mass variables and constraints
  comnext = prog.NewContinuousVariables(3, "comnext")
  comdotnext = prog.NewContinuousVariables(3, "comdotnext")
  comddot = prog.NewContinuousVariables(3, "comddot")

  constraint3 = prog.AddLinearConstraint((halfspace_polyhedral.A() @ comnext[:2])[0] <= 0.8*halfspace_polyhedral.b().T[0])
  constraint3.evaluator().set_description("CoM inside foot support constraint 3")
  constraint4 = prog.AddLinearConstraint((halfspace_polyhedral.A() @ comnext[:2])[1] <= 0.8*halfspace_polyhedral.b().T[1])
  constraint4.evaluator().set_description("CoM inside foot support constraint 4")
  constraint5 = prog.AddLinearConstraint((halfspace_polyhedral.A() @ comnext[:2])[2] <= 0.8*halfspace_polyhedral.b().T[2])
  constraint5.evaluator().set_description("CoM inside foot support constraint 5")
  constraint6 = prog.AddLinearConstraint((halfspace_polyhedral.A() @ comnext[:2])[3] <= 0.8*halfspace_polyhedral.b().T[3])
  constraint6.evaluator().set_description("CoM inside foot support constraint 6")

  # # CoM dynamics
  com_in_world = plant.CalcCenterOfMassPositionInWorld(plant_context)
  j_com = plant.CalcJacobianCenterOfMassTranslationalVelocity(
    plant_context,
    JacobianWrtVariable.kV,
    plant.world_frame(),
    plant.world_frame(),
  )
  
  integration_factor = 10
  constraint8 = prog.AddConstraint(eq(comnext, com_in_world + integration_factor * timeStep * j_com @ v))
  constraint8.evaluator().set_description("CoMnext dynamics constraint 8")
  constraint9 = prog.AddConstraint(
    eq(comdotnext, j_com @ v + timeStep * comddot)
  )
  constraint9.evaluator().set_description("CoMdotnext dynamics constraint 9")
  constraint10 = prog.AddConstraint(
    eq(
      total_mass * comddot,
      sum(contact_force[i] for i in range(4))
      + total_mass * gravity,
    )
  )
  constraint10.evaluator().set_description("CoMdotdot dynamics constraint 10")

  #####################
  # comddot = = J𝑠_v_ACcm ⋅ ṡ + abias_ACcm.

  ad_plant = plant.ToAutoDiffXd()
  ad_context = ad_plant.CreateDefaultContext()
  context = plant.CreateDefaultContext()
  def CoM_Acceleration(vars):
    vdot, comddot = np.split(vars, [nv])

    Jdotv = plant.CalcBiasCenterOfMassTranslationalAcceleration(
      plant_context,
      JacobianWrtVariable.kV,
      plant.world_frame(),
      plant.world_frame(),
    )
    return ((j_com @ vdot + Jdotv) - comddot)

  # prog.AddConstraint(eq(vnext, v + timeStep * vdot))
  epsilon = 0
  CoM_Acceleration_constraint = prog.AddConstraint(
    partial(CoM_Acceleration),
    lb=-epsilon*np.ones(3),
    ub=epsilon*np.ones(3),
    vars=np.concatenate((vdot, comddot)),
  )
  CoM_Acceleration_constraint.evaluator().set_description("CoM Acceleration constraint")
  # prog.AddQuadraticErrorCost(np.diag(v_cost), [0] * nv, vnext)
  #####################

#   # H_f vdot − J_f^T * λ = −C_f
  ad_foot_frame = [
    ad_plant.GetFrameByName("front_left_foot_center"),
    ad_plant.GetFrameByName("front_right_foot_center"),
    ad_plant.GetFrameByName("rear_left_foot_center"),
    ad_plant.GetFrameByName("rear_right_foot_center"),
  ]
  ad_leg_frame = [
    ad_plant.GetFrameByName("front_left_lower_leg"),
    ad_plant.GetFrameByName("front_right_lower_leg"),
    ad_plant.GetFrameByName("rear_left_lower_leg"),
    ad_plant.GetFrameByName("rear_right_lower_leg"),
  ]
  leg_frame = [
    plant.GetFrameByName("front_left_lower_leg"),
    plant.GetFrameByName("front_right_lower_leg"),
    plant.GetFrameByName("rear_left_lower_leg"),
    plant.GetFrameByName("rear_right_lower_leg"),
  ]
  ad_body_frame = ad_plant.GetFrameByName("body")
  def unactuated_dynamics(vars):
    qv, vdot, contact_force = np.split(vars, [nq+nv, nq+2*nv])
    contact_force = contact_force.reshape(3, 4, order="F")
    if isinstance(vars[0], AutoDiffXd):
      ad_plant.SetPositionsAndVelocities(ad_context, qv)
      ad_Cv_f = ad_plant.CalcBiasTerm(ad_context)[:6]
      ad_H_f = ad_plant.CalcMassMatrix(ad_context)[:6]
      ad_tau_q = ad_plant.CalcGravityGeneralizedForces(ad_context)[:6]
      # print(f"ad_tau_q shape = {ad_tau_q.shape}")
      # print(f"ad_tau_q = {ExtractValue(ad_tau_q)}")
      dynamics_output = ad_H_f @ vdot + ad_Cv_f - ad_tau_q
      for i in range(4):
        ad_foot_pose_in_world_var = ad_plant.CalcPointsPositions(
          ad_context, ad_foot_frame[i], [0, 0, 0], ad_plant.world_frame()
        )
        ad_foot_pose_in_leg_var = ad_plant.CalcPointsPositions(
          ad_context, ad_foot_frame[i], [0, 0, 0], ad_leg_frame[i]
        )
        ad_J_W_WF = ad_plant.CalcJacobianTranslationalVelocity(
          ad_context,
          JacobianWrtVariable.kV,
          ad_plant.world_frame(),
          ad_foot_pose_in_world_var,
          ad_plant.world_frame(),
          ad_plant.world_frame(),
        )
        ad_J_L_WF = ad_plant.CalcJacobianTranslationalVelocity(
          ad_context,
          JacobianWrtVariable.kV,
          ad_leg_frame[i],
          ad_foot_pose_in_leg_var,
          ad_plant.world_frame(),
          ad_plant.world_frame(),
        )
        # ad_plant.get_applied_spatial_force_input_port().FixValue(ad_context, )
        dynamics_output = dynamics_output - (ad_J_W_WF - ad_J_L_WF)[:,:6].T @ contact_force[:, i]
      return dynamics_output
    else:
      plant.SetPositionsAndVelocities(context, qv)
      Cv_f = plant.CalcBiasTerm(context)[:6]
      H_f = plant.CalcMassMatrix(context)[:6]
      tau_q = plant.CalcGravityGeneralizedForces(context)[:6]
      dynamics_output = H_f @ vdot + Cv_f - tau_q 
      for i in range(4):
        foot_pose_in_world_var = plant.CalcPointsPositions(
          context, foot_frame[i], [0, 0, 0], plant.world_frame()
        )
        foot_pose_in_leg_var = plant.CalcPointsPositions(
          context, foot_frame[i], [0, 0, 0], leg_frame[i]
        )
        J_W_WF = plant.CalcJacobianTranslationalVelocity(
          context,
          JacobianWrtVariable.kV,
          plant.world_frame(),
          foot_pose_in_world_var,
          plant.world_frame(),
          plant.world_frame(),
        )
        J_L_WF = plant.CalcJacobianTranslationalVelocity(
          context,
          JacobianWrtVariable.kV,
          leg_frame[i],
          foot_pose_in_leg_var,
          plant.world_frame(),
          plant.world_frame(),
        )

        dynamics_output = dynamics_output - (J_W_WF - J_L_WF)[:,:6].T @ contact_force[:,i]

      return dynamics_output

  Fn = np.concatenate([contact_force[i] for i in range(4)])
  epsilon = 0
  constraint_unactuated_dynamics = prog.AddConstraint(
    partial(unactuated_dynamics),
    lb=-epsilon*np.ones(6),
    ub=epsilon*np.ones(6),
    vars=np.concatenate((q, v, vdot, Fn)),
  )
  constraint_unactuated_dynamics.evaluator().set_description("Unactuated dynamics Constraint")
  
  # def visualization_callback(vars):
  #   q, v, comnext, comdotnext, comddot = np.split(vars, [nq, nq + nv, nq + nv + 3, nq + nv + 6])
    # print(f"q diff = {q - qf}")
    # print(f"v diff = {v}")
    # print(f"j_com @ q = {(j_com @ v)}")
    # print(f"comnext = {comnext}")
    # print(f"comdotnext = {comdotnext}")
    # print(f"comddot = {comddot}")
    # print(f"comnext calculated = {com_in_world + integration_factor*timeStep * (j_com @ v)}")
    # print(f"CoM condition: {(halfspace_polyhedral.A() @ (com_in_world + integration_factor*timeStep * (j_com @ v))[:2]) - 0.8*halfspace_polyhedral.b().T}")
    # print(f"comdotnext calculated = {j_com @ v + integration_factor*timeStep * comddot}")


  # prog.AddVisualizationCallback(visualization_callback, np.concatenate((q, v, comnext, comdotnext, comddot)))


  snopt = SnoptSolver().solver_id()
  prog.SetSolverOption(
    snopt, "Iterations Limits", 1e5 if running_as_notebook else 1
  )
  prog.SetSolverOption(
    snopt, "Major Iterations Limit", 200 if running_as_notebook else 1
  )
  prog.SetSolverOption(snopt, "Major Feasibility Tolerance", 5e-6)
  prog.SetSolverOption(snopt, "Major Optimality Tolerance", 1e-4)
  prog.SetSolverOption(snopt, "Superbasics limit", 2000)
  prog.SetSolverOption(snopt, "Linesearch tolerance", 0.9)
  result = Solve(prog)
  print("Is success: " + str(result.is_success()))
  print(f"optimization status: {result.get_solution_result()}")
  q_sol = result.GetSolution(q)
  v_sol = result.GetSolution(v)
  comddot_sol = result.GetSolution(comddot)
  comdotnext_sol = result.GetSolution(comdotnext)
  comnext_sol = result.GetSolution(comnext)
  contact_force_sol = result.GetSolution(contact_force)
  infeasible_constraints = result.GetInfeasibleConstraints(prog)
  for c in infeasible_constraints:
    print(f"infeasible constraint: {c}")

  # print(f"comdotnext_sol={comdotnext_sol.T}")    
  # print(f"v_sol={v_sol.T}")
  # print(f"j_com @ v_sol = {(j_com @ v_sol)}")
  # print(f"CoM in a box constraint: {(halfspace_polyhedral.A() @ comnext_sol[:2]) - 0.8*halfspace_polyhedral.b().T}\n")
  # print(f"from plant contest com = {com_in_world}")
  # print(f"comnext_sol={comnext_sol.T}")

  # print(f"total_mass = {total_mass}")
  # print(f"q_diff={q_sol.T - qf}")
  # print(f"v_sol={v_sol.T}")

  # print(f"contact_force_sol[0] = {contact_force_sol[0].T}")
  # print(f"contact_force_sol[1] = {contact_force_sol[1].T}")
  # print(f"contact_force_sol[2] = {contact_force_sol[2].T}")
  # print(f"contact_force_sol[3] = {contact_force_sol[3].T}")
  # print(f"comddot_sol={comddot_sol.T}")

  # print(f"comddot constraint = {(total_mass * comddot_sol) -  sum(contact_force_sol[i] for i in range(4)) - total_mass * gravity}")

  return result, q, v

The following function is the main function that creates the diagram , the simulator, set the time step and calls `qp_optimization_continuous` periodically

In [8]:
def qp_dp_continuous_control():
  print(f"current path: {os.getcwd()}")
  current_os_path = os.getcwd()
  timeStep = 1e-3
  builder = DiagramBuilder()
  plant, scene_graph = AddMultibodyPlantSceneGraph(builder, timeStep)
  parser = Parser(plant)
  (spot,) = parser.AddModelsFromUrl(
      f"file://{current_os_path}/spot_description/urdf/spot.urdf"
  )
  (spot_arm,) = parser.AddModelsFromUrl(
      f"file://{current_os_path}/spot_description/urdf/spot_arm.urdf"
  )
  (ground,) = parser.AddModelsFromUrl(
      f"file://{current_os_path}/spot_description/urdf/ground.urdf"
  )
  (ball,) = parser.AddModelsFromString(weight_ball, 'urdf')
  
  weld_joint_arm = plant.WeldFrames(plant.GetFrameByName("body", spot), plant.GetFrameByName("arm_base", spot_arm), RigidTransform(RollPitchYaw(1.570796326794897, 0, 0), [0.29, 0.0, 0.11]))
  weld_joint_ball = plant.WeldFrames(plant.GetFrameByName("finger", spot_arm), plant.GetFrameByName("ball", ball), RigidTransform(RollPitchYaw(0, 0, 0), [-0.07, 0.0, -0.08]))
  
  plant.Finalize()

  control, S = create_pid_controller(builder, plant)

  builder.Connect(
    plant.get_state_output_port(), control.get_input_port_estimated_state()
  )
  builder.Connect(
    control.get_output_port(), plant.get_actuation_input_port()
  )

  visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

  state_logger = LogVectorOutput(
      plant.get_state_output_port(), builder
  )
  input_logger = LogVectorOutput(control.get_output_port(), builder)

  diagram = builder.Build()
  # context = diagram.CreateDefaultContext()
  simulator = Simulator(diagram)
  
  # display(SVG(pydot.graph_from_dot_data(
  # diagram.GetGraphvizString(max_depth=2))[0].create_svg()))

  sim_context = simulator.get_mutable_context()
  plant_context = plant.GetMyContextFromRoot(sim_context)
  context = simulator.get_mutable_context()
  set_home(plant, plant_context)
  # diagram.ForcedPublish(sim_context)
  q0 = plant.GetPositions(plant_context)
  v0 = plant.GetVelocities(plant_context)
  PositionView = namedview(
      "Positions", plant.GetPositionNames(always_add_suffix=False, add_model_instance_prefix=True)
  )

  VelocityView = namedview(
      "Velocities",
      plant.GetVelocityNames(always_add_suffix=False, add_model_instance_prefix=True),
  )
  
  ActuationView = namedview(
    "Actuation",
    plant.GetActuatorNames(add_model_instance_prefix=False),
  )

  foot_frame = [
    plant.GetFrameByName("front_left_foot_center"),
    plant.GetFrameByName("front_right_foot_center"),
    plant.GetFrameByName("rear_left_foot_center"),
    plant.GetFrameByName("rear_right_foot_center"),
  ]

  foot_position = [plant.CalcPointsPositions(
        plant_context, foot_frame[idx], [0, 0, 0], plant.world_frame()
    ) for idx in range(4)]
  # for idx in range(4):
  #   print(f"{foot_frame[idx].name()} position is {foot_position[idx]} | ")
  # print("\n")
  foot_pose_in_world = np.zeros((4,3))
  for idx in range(4):
    foot_pose_in_world[idx, :] = plant.CalcPointsPositions(
      plant_context, foot_frame[idx], [0, 0, 0], plant.world_frame()
    )[:,0]
  print(f"foot_pose_in_world: {foot_pose_in_world[:,:2]}")
  com_pose_in_world = plant.CalcCenterOfMassPositionInWorld(plant_context)
  # print(f"foot_pose_in_world: {foot_pose_in_world}")
  # print(f"com_pose_in_world: {com_pose_in_world}")
  meshcat.SetObject("CoM", Sphere(0.03), Rgba(0, 0, 1, 1))
  for i in range(4):
    meshcat.SetObject(f"foot_{i}", Sphere(0.03), Rgba(1, 0, 0, 1)) 

  com_pose_in_world[2] = 0
  meshcat.SetTransform(
      "CoM", RigidTransform(com_pose_in_world)
  )

  for i in range(4):
    meshcat.SetTransform(f"foot_{i}", RigidTransform(foot_pose_in_world[i]))
  
  # print(f"full plan positions: {plant.GetPositions(plant_context).shape}")
  # print(f"full plan velocities: {plant.GetVelocities(plant_context).shape}")
  # print(f'plant position q0: {q0}\n')
  # print(f'plant position q0 length: {len(q0)}\n')
  # print(f'body frame position: {body_frame}\n')
  # print(f'positionFields: {PositionView.get_fields()}\n')
  # print(f'velocityFields: {VelocityView.get_fields()}\n')
  # print(f'mu = {mu}, robot total mass = {total_mass}, gravity = {gravity}')
  # print(result.get_solver_id().name())
  # print("Is success: " + str(result.is_success()))
  # print("Solution(q): " + str(result.GetSolution(q)))
  # print("Solution(q) shape: " + str(result.GetSolution(q).shape))
  # print("Solution(v): " + str(result.GetSolution(v)))
  # print("Solution(v) shape: " + str(result.GetSolution(v).shape))
  # print("Optimal Cost: " + str(result.get_optimal_cost()))
  # print(f"result: {result}")
  # print(f"Solution Result: {result.get_solution_result().kSolutionFound}")

  result, q, v = qp_optimization_continuous(sim_context, timeStep, plant, q0, v0, q0)
  if result.is_success():
    q_sol = result.GetSolution(q)
    v_sol = result.GetSolution(v)
    position_and_velocity = np.zeros((q_sol.shape[0] + v_sol.shape[0]))
    position_and_velocity[:q_sol.shape[0]] = q_sol[:]
    position_and_velocity[q_sol.shape[0]:] = v_sol[:]
    # print(f"q_sol: {q_sol}")
    # print(f"commanded difference: {q0-q_sol}")
    # print(f"v_sol: {v_sol}")
    x0 = S @ position_and_velocity #plant.get_state_output_port().Eval(plant_context)

    print(f"x0: {x0}")
    control.get_input_port_desired_state().FixValue(
        control.GetMyContextFromRoot(sim_context), x0
    )
  else:
    x0 = S @ plant.get_state_output_port().Eval(plant_context)
    control.get_input_port_desired_state().FixValue(
        control.GetMyContextFromRoot(sim_context), x0
    )

  simulator.set_target_realtime_rate(1.0 if running_as_notebook else 0)
  visualizer.StartRecording()
  q0_view = PositionView(q0)
  v0_view = VelocityView(v0)
  qf_view = copy.deepcopy(q0_view)
  servo_rpm = 120
  add_spot_arm_sliders(x0)
  while meshcat.GetButtonClicks("Stop") < 1:
    shoulder_roll_cmd = meshcat.GetSliderValue("shoulder_roll")
    shoulder_pitch_cmd = meshcat.GetSliderValue("shoulder_pitch")
    elbow_pitch_cmd = meshcat.GetSliderValue("elbow_pitch")
    lower_arm_roll_cmd = meshcat.GetSliderValue("lower_arm_roll")
    wrist_pitch_cmd = meshcat.GetSliderValue("wrist_pitch")
    wrist_roll_cmd = meshcat.GetSliderValue("wrist_roll")
    finger_pitch_cmd = meshcat.GetSliderValue("finger_pitch")

    foot_pose_in_world_display = np.zeros((4,3))
    for idx in range(4):
      foot_pose_in_world_display[idx, :] = plant.CalcPointsPositions(
        plant_context, foot_frame[idx], [0, 0, 0], plant.world_frame()
      )[:,0]
    for i in range(4):
      meshcat.SetTransform(f"foot_{i}", RigidTransform(foot_pose_in_world_display[i]))

    com_pose_in_world = plant.CalcCenterOfMassPositionInWorld(plant_context)
    # print(f"from sim context com = {com_pose_in_world}")
    com_pose_in_world[2] = 0
    meshcat.SetTransform(
        "CoM", RigidTransform(com_pose_in_world)
    )
    
    qf_view = copy.deepcopy(q0_view)
    command_increment = ((servo_rpm * 2 * np.pi)/60) * timeStep

    qf_view = set_qf(qf_view, command_increment, shoulder_roll_cmd, shoulder_pitch_cmd, elbow_pitch_cmd, lower_arm_roll_cmd, wrist_pitch_cmd, wrist_roll_cmd, finger_pitch_cmd)
    # print(f"position commanded difference: {q0_view[:]-qf_view[:]}")
    result, q, v = qp_optimization_continuous(sim_context, timeStep, plant, q0_view[:], v0_view[:], qf_view[:])
    # print("Is success: " + str(result.is_success()))

    if result.is_success():
      q_sol = result.GetSolution(q)
      v_sol = result.GetSolution(v)
      position_and_velocity = np.zeros((q_sol.shape[0] + v_sol.shape[0]))
      position_and_velocity[:q_sol.shape[0]] = q_sol[:]
      position_and_velocity[q_sol.shape[0]:] = v_sol[:]
      # print(f"q_sol: {q_sol}")
      # print(f"q0_view: {q0_view[:]}")
      # print(f"qf_view: {qf_view[:]}")
      # print(f"position commanded difference: {q_sol-qf_view[:]}")
      # print(f"position_and_velocity: {position_and_velocity}")
      x0 = S @ position_and_velocity
      
      # print(f"x0: {x0}")
      control.get_input_port_desired_state().FixValue(
          control.GetMyContextFromRoot(sim_context), x0
      )
      
      q0_view = PositionView(copy.deepcopy(q_sol))
      v0_view = VelocityView(copy.deepcopy(v_sol))

      
    simulator.AdvanceTo(simulator.get_context().get_time() + timeStep)
  # # display(SVG(pydot.graph_from_dot_data(
  # # diagram.GetGraphvizString(max_depth=2))[0].create_svg()))


  visualizer.PublishRecording()
  meshcat.DeleteAddedControls()
  meshcat.Delete("CoM")
  for i in range(4):
    meshcat.Delete(f"foot_{i}") 

  # nq = plant.num_positions()
  # nv = plant.num_velocities()
  # na = plant.num_actuators()
  # positionFields = PositionView.get_fields()
  # velocityFields = VelocityView.get_fields()
  # actuationFields = ActuationView.get_fields()
  # # Plot the results
  # stateLog = state_logger.FindLog(context)
  # # print(f'stateLog data size: {stateLog.data().shape}')
  # controllerLog = input_logger.FindLog(context)
  # for i in range(plant.num_joints()):
  #   joint = plant.get_joint(JointIndex(i))
  #   if joint.num_positions() != 1:
  #     continue
  #   idx = actuationFields.index(joint.name())
  #   jdx = joint.position_start()
  #   wdx = nq + joint.velocity_start()
  #   plt.figure()
  #   plt.subplot(3, 1, 1)
  #   refPosition = x0[idx] * np.ones(len(stateLog.sample_times()))
  # #   # print(f'refPosition: {refPosition}\n')
  # #   # print(f'stateLog.data()[jdx, :].transpose(): {stateLog.data()[jdx, :].transpose()}\n')
  #   plt.plot(stateLog.sample_times(), (180/np.pi)*stateLog.data()[jdx, :].transpose())
  #   plt.plot(stateLog.sample_times(), (180/np.pi)*refPosition)
  #   plt.legend(["position", "desired position"])
  #   plt.ylabel("Position")
  #   plt.title(actuationFields[idx])
  #   plt.grid(True)
  #   plt.subplot(3, 1, 2)
  #   refVelocity = x0[idx+na] * np.ones(len(stateLog.sample_times()))
  # #   # print(f'refVelocity: {refVelocity}\n')
  # #   # print(f'stateLog.data()[wdx, :].transpose(): {stateLog.data()[jdx, :].transpose()}\n')
  #   plt.plot(stateLog.sample_times(), (180/np.pi)*stateLog.data()[wdx, :].transpose())
  #   plt.plot(stateLog.sample_times(), (180/np.pi)*refVelocity)
  #   plt.legend(["velocity", "desired velocity"])
  #   plt.ylabel("Velocity")
  #   plt.grid(True)
  #   plt.subplot(3, 1, 3)
  #   plt.plot(controllerLog.sample_times(), controllerLog.data()[idx, :].transpose())
  #   plt.legend(["Actuation"])
  #   plt.xlabel('t')
  #   plt.ylabel(actuationFields[idx])
  #   plt.grid(True)
  #   plt.savefig("plots/" + actuationFields[idx] + ".jpg")
    # display(mpld3.display())

qp_dp_continuous_control()

current path: /Users/namirjawdat/Projects/underactuatedFinalProject/QPInverseDynamicsForSpotWithAnArm
Joint[19] weld is not considered
Joint[20] body_welds_to_arm_base is not considered
Joint[21] finger_welds_to_ball is not considered
Joint[22] $world_body is not considered
foot_pose_in_world: [[ 0.4141103   0.16275   ]
 [ 0.4141103  -0.16725   ]
 [-0.32703284  0.16275   ]
 [-0.32703284 -0.16725   ]]
Is success: True
optimization status: SolutionResult.kSolutionFound
x0: [ 4.54724758e-03 -6.00281070e-01  1.54990614e+00  4.54568411e-03
 -6.00341024e-01  1.54997272e+00  4.57635712e-03 -9.00060263e-01
  1.54997524e+00  4.57484839e-03 -9.00133894e-01  1.55005151e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00 -7.41162169e-07
 -7.45582160e-07  2.79252639e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.

Is success: True
optimization status: SolutionResult.kSolutionFound
Is success: True
optimization status: SolutionResult.kSolutionFound
Is success: True
optimization status: SolutionResult.kSolutionFound
Is success: True
optimization status: SolutionResult.kSolutionFound
Is success: True
optimization status: SolutionResult.kSolutionFound
Is success: True
optimization status: SolutionResult.kSolutionFound
Is success: True
optimization status: SolutionResult.kSolutionFound
Is success: True
optimization status: SolutionResult.kSolutionFound
Is success: True
optimization status: SolutionResult.kSolutionFound
Is success: True
optimization status: SolutionResult.kSolutionFound
Is success: True
optimization status: SolutionResult.kSolutionFound
Is success: True
optimization status: SolutionResult.kSolutionFound
Is success: True
optimization status: SolutionResult.kSolutionFound
Is success: True
optimization status: SolutionResult.kSolutionFound
Is success: True
optimization status: SolutionRe